In [51]:
import numpy as np
import pandas as pd
import requests as rq
import html5lib
import lxml

from selenium import webdriver
from bs4 import BeautifulSoup as bs

**Note: use Chrome.** Remember to get geckodriver for Firefox or [ChromeDriver](https://sites.google.com/a/chromium.org/chromedriver/home) for Chrome. Also need lxml and html5lib but installed don't need to import. pandas read_html prioritized lxml then html5lib.

Some protips for getting xpaths [here](http://thiagomarzagao.com/2013/11/12/webscraping-with-selenium-part-1/).

Form inputs:

In [115]:
employer_name = 'aqr'
job_name = ''
city_name = ''
year_name = 'All Years' # has dropdown until 2012

# input data dictionary
input_fields = {'employer': {'user_input': employer_name,
                            'xpath': '//*[@id="employer"]',
                            'type': 'text'},
               'job': {'user_input': job_name,
                      'xpath': '//*[@id="job"]',
                      'type': 'text'},
               'city': {'user_input': city_name,
                       'xpath': '//*[@id="city"]',
                       'type': 'text'},
               'year': {'user_input': year_name,
                       'xpath': '//select[@name="year"]',
                       'type': 'select'}}

### Method 1: Direct URL access to table

Direct paste into URL:

In [116]:
search_url = (
    'http://h1bdata.info/index.php?em=' + employer_name + 
    '&job=' + job_name +
    '&city=' + city_name +
    '&year=' + year_name
    )
search_url = search_url.replace(' ', '+')
print search_url

http://h1bdata.info/index.php?em=aqr&job=&city=&year=All+Years


In [117]:
search_page = rq.get(search_url)
page_soup = bs(search_page.content, 'lxml')

### Method 2: Enter form inputs through Selenium Webdriver

**Note:** for faster access you can use direct request instead of WebDriver. In fact, even within the realm of form-input methods, using Beautifulsoup might even be faster. This is just for practice although it is more generalizable for sites that generate using JavaScript. A useful future feature would be to scrape the autocompletion entries for the employer names as well.

Initialize WebDriver:

In [118]:
home_url = 'http://h1bdata.info/'
browser = webdriver.Chrome()

In [119]:
browser.get(home_url)

for field in input_fields:
    curr_elem = browser.find_element_by_xpath(input_fields[field]['xpath'])
    
    # dropdown lists
    if input_fields[field]['type'] == 'select':
        
        # while scraping, record finite values of dropdowns for future use
        option_values = curr_elem.find_elements_by_tag_name('option')
        input_fields[field]['values'] = [option.get_attribute('value') 
                                         for option in option_values]
        
        print field, "values: \n", input_fields[field]['values']
        
        # select user input option
        option_xpath = (input_fields[field]['xpath'] + 
                             '/option[@value="' + 
                             input_fields[field]['user_input'] + 
                             '"]'
                       )
        curr_elem_option = browser.find_element_by_xpath(option_xpath)
        curr_elem_option.click()
    
    # text input lists
    elif input_fields[field]['type'] == 'text':
        curr_elem.clear()
        curr_elem.send_keys(input_fields[field]['user_input'])
    
    # input field missing
    else:
        print "Unaccounted Field:", field
    
    
browser.find_element_by_css_selector('button[type="submit"]').click()

year values: 
[u'All Years', u'2016', u'2015', u'2014', u'2013', u'2012']


## Get Tables

Scrape table from direct URL and BeautifulSoup:

In [123]:
out_table_direct = page_soup.find('table')
df_table = pd.read_html(out_table_direct.prettify())[0]
df_table.head()

,EMPLOYER,JOB TITLE,BASE SALARY,LOCATION,SUBMIT DATE,START DATE,CASE STATUS
0,AQR CAPITAL MANAGEMENT LLC,"ALGORITHMIC DEVELOPER, ASSOCIATE",130000,"GREENWICH, CT",06/09/2014,07/01/2014,CERTIFIED
1,AQR CAPITAL MANAGEMENT LLC,ANALYST,75000,"GREENWICH, CT",06/06/2013,06/17/2013,CERTIFIED
2,AQR CAPITAL MANAGEMENT LLC,ANALYST,90000,"GREENWICH, CT",07/11/2016,07/19/2016,CERTIFIED
3,AQR CAPITAL MANAGEMENT LLC,"ANALYST, SECURITY MASTER SPECIALIST",90000,"GREENWICH, CT",02/04/2015,02/23/2015,CERTIFIED
4,AQR CAPITAL MANAGEMENT LLC,"ANALYST, STRATEGY",100000,"GREENWICH, CT",05/31/2016,08/15/2016,CERTIFIED


Scrape table from WebDriver window:

In [124]:
out_table = browser.find_element_by_tag_name('table')
df_table = pd.read_html(out_table.get_attribute('outerHTML'), flavor='lxml')[0]
df_table.head()

,EMPLOYER,JOB TITLE,BASE SALARY,LOCATION,SUBMIT DATE,START DATE,CASE STATUS
0,AQR CAPITAL MANAGEMENT LLC,"ALGORITHMIC DEVELOPER, ASSOCIATE",130000,"GREENWICH, CT",06/09/2014,07/01/2014,CERTIFIED
1,AQR CAPITAL MANAGEMENT LLC,ANALYST,75000,"GREENWICH, CT",06/06/2013,06/17/2013,CERTIFIED
2,AQR CAPITAL MANAGEMENT LLC,ANALYST,90000,"GREENWICH, CT",07/11/2016,07/19/2016,CERTIFIED
3,AQR CAPITAL MANAGEMENT LLC,"ANALYST, SECURITY MASTER SPECIALIST",90000,"GREENWICH, CT",02/04/2015,02/23/2015,CERTIFIED
4,AQR CAPITAL MANAGEMENT LLC,"ANALYST, STRATEGY",100000,"GREENWICH, CT",05/31/2016,08/15/2016,CERTIFIED


Data exploration:

In [122]:
df_table.LOCATION.value_counts()

GREENWICH, CT    167
Name: LOCATION, dtype: int64